<a href="https://colab.research.google.com/github/esh04/Detoxifier/blob/main/Detox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 58.1 MB/s 
     |████████████████████████████████| 163 kB 66.5 MB/s 


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import re

# load tokenizer and model weights
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

# # # prepare the input
# batch = tokenizer.encode('you are amazing', return_tensors='pt')
# print(batch)
# print(model(batch))
# output = torch.nn.functional.softmax(model(batch).logits, dim = -1)
# # inference
# print(output)

# batch = tokenizer.encode('these clowns are useless', return_tensors='pt')
# print(batch)

# # print(model.predict('you are of no use at all'))

# print(model(batch))
# output = torch.nn.functional.softmax(model(batch).logits, dim = -1)
# print(float(output[0][1]))

# batch = tokenizer.encode('are', return_tensors='pt')
# print(batch)


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[   0, 6968,   32, 2770,    2]])
SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6567, -4.9115]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[9.9993e-01, 6.9912e-05]], grad_fn=<SoftmaxBackward0>)
tensor([[    0, 29902, 21068,    29,    32, 23584,     2]])
SequenceClassifierOutput(loss=None, logits=tensor([[-3.7082,  3.5778]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0.9993153810501099
tensor([[   0, 1322,    2]])


In [ ]:
def mask(sentence):
  # for i in range(len(sentence)):
  #   if (not sentence[i].isalpha()):
  #     sentence[i]=' '
  sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
  
  sentence = ' ' + sentence + ' '

  sentence = sentence.split(' ')

  # true_sent = []
  # for i in sentence:
  #   true_sent.append(i)
  # true_sent = sentence
  true_sent = sentence.copy()

  # print(true_sent)
  masked_sentence = ''
  min_tox = 1
  while True:
    for i in range(len(sentence)):

      # print('1:',i,sentence)
      sentence[i] = '[MASK]'
      # print('2:',i,sentence)
      sentence = ' '.join(sentence)
      batch = tokenizer.encode(sentence, return_tensors='pt')
      output = torch.nn.functional.softmax(model(batch).logits, dim = -1)
      toxic_score = float(output[0][1])
      if min_tox > toxic_score:
        min_tox = toxic_score
        masked_sentence = sentence

      # print('nvwubrb',true_sent)
      sentence = true_sent.copy()

    
    if min_tox < 0.25:
      break
    
    true_sent = masked_sentence
    true_sent = true_sent.split(' ')
  
  return masked_sentence




In [ ]:
sent = mask("hey loser , try this get a fucking life and stay out of mine , which you know nothing about ")
batch = tokenizer.encode(sent, return_tensors ='pt')
output = torch.nn.functional.softmax(model(batch).logits,dim = -1)
print(output[0][1])
print(sent)

tensor(0.0528, grad_fn=<SelectBackward0>)
 hey [MASK]   try this get a [MASK] life and stay out of mine   which you know nothing about  
